In [1]:
# importing dependencies here
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from splinter import Browser
import pandas as pd
from pprint import pprint

%load_ext nb_black

<IPython.core.display.Javascript object>

### NASA Mars News

In [2]:
# url for NASA's news web page
URL = "https://mars.nasa.gov/news/"

<IPython.core.display.Javascript object>

In [3]:
# using selenium to scrape the url as requests didn't return the latest news
driver = webdriver.Firefox()  # opening the browser
driver.get(URL)  # navigating to the web page
html = driver.page_source
driver.implicitly_wait(20)
driver.close()

soup = BeautifulSoup(html, "lxml")

<IPython.core.display.Javascript object>

In [4]:
# extracting the latets news title and teaser
latest_news = soup.find("div", class_="list_text")
latest_news_title = latest_news.find("div", class_="content_title").text
latest_news_teaser = latest_news.find("div", class_="article_teaser_body").text
print(f"Latest Mars News Title: {latest_news_title}")
print(f"Latest Mars News Teaser: {latest_news_teaser}")


Latest Mars News Title: The MarCO Mission Comes to an End
Latest Mars News Teaser: The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.


<IPython.core.display.Javascript object>

### JPL Mars Space Images - Featured Image

In [5]:
# url for NASA's current featured image of Mars
IMG_URL = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

<IPython.core.display.Javascript object>

In [6]:
# extracting the html text and creating the soup object
response = requests.get(IMG_URL)
html = response.text
soup = BeautifulSoup(html, "lxml")
type(soup)

bs4.BeautifulSoup

<IPython.core.display.Javascript object>

In [7]:
# extracting the featured the image url and then the featured full image using that url
featured_image_url = (
    "https://www.jpl.nasa.gov"
    + soup.find("a", class_="button fancybox")["data-fancybox-href"]
)
print(f"Featured Image url is: {featured_image_url}")


Featured Image url is: https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14254_ip.jpg


<IPython.core.display.Javascript object>

### Mars Weather

In [8]:
WEATHER_URL = "https://twitter.com/marswxreport?lang=en"

<IPython.core.display.Javascript object>

In [9]:
# extracting the html text and creating the soup object
response = requests.get(WEATHER_URL)
html = response.text
soup = BeautifulSoup(html, "html.parser")
type(soup)

bs4.BeautifulSoup

<IPython.core.display.Javascript object>

In [10]:
# print the html to get the correct twitter class name. What we see on inspecting twitter are junk class values.
twitter_weather_para = soup.find(
    "p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"
)
for para in twitter_weather_para:
    mars_weather = para
    break

print(f"Mars Weather: {mars_weather}")

Mars Weather: InSight sol 447 (2020-02-28) low -92.6ºC (-134.6ºF) high -11.1ºC (12.0ºF)
winds from the SSW at 5.8 m/s (13.1 mph) gusting to 20.2 m/s (45.2 mph)
pressure at 6.30 hPa


<IPython.core.display.Javascript object>

### Mars Facts

In [11]:
FACTS_URL = "http://space-facts.com/mars/"

<IPython.core.display.Javascript object>

In [12]:
reponse = requests.get(FACTS_URL).text
mars_facts = pd.read_html(reponse)[0]
print(mars_facts)

                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers


<IPython.core.display.Javascript object>

In [13]:
import pandas as pd

mars_facts_df = (
    pd.DataFrame(mars_facts)
    .rename(columns={0: "Description", 1: "Value"})
    .set_index("Description")
)

mars_facts_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


<IPython.core.display.Javascript object>

In [14]:
mars_facts = mars_facts_df.to_html(index=True, header=True)
mars_facts = mars_facts.replace("\n", "")

mars_facts

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

<IPython.core.display.Javascript object>

### Mars Hemispheres

In [15]:
HEMISPHERE_URL = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

<IPython.core.display.Javascript object>

In [16]:
browser = Browser(
    "firefox"
)  # Note: if you don’t provide any driver argument to the Browser function, firefox will be used
browser.visit(HEMISPHERE_URL)
html = browser.html

soup = BeautifulSoup(html, "lxml")
type(soup)

bs4.BeautifulSoup

<IPython.core.display.Javascript object>

In [17]:
page = soup.find("div", class_="result-list")
page_items = page.find_all("div", class_="item")
# print(page_items)

<IPython.core.display.Javascript object>

In [18]:
HEMISPHERE_PARTIAL_URL = "https://astrogeology.usgs.gov"
hemisphere_image_urls = []

for item in page_items:

    hemisphere_dict = {}

    url = item.find("a")["href"]
    #     print(HEMISPHERE_PARTIAL_URL + url)

    browser.visit(HEMISPHERE_PARTIAL_URL + url)
    new_html = browser.html
    soup = BeautifulSoup(new_html, "lxml")

    hemisphere_dict["title"] = soup.find("h2", class_="title").text
    image = soup.find("div", class_="downloads")
    hemisphere_dict["img_url"] = image.find("a")["href"]

    hemisphere_image_urls.append(hemisphere_dict)

pprint(hemisphere_image_urls)
browser.quit()

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]


<IPython.core.display.Javascript object>

In [19]:
hemisphere_df = pd.DataFrame(hemisphere_image_urls)

<IPython.core.display.Javascript object>

In [20]:
hemisphere_df

,title,img_url
0,Cerberus Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
1,Schiaparelli Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
2,Syrtis Major Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...
3,Valles Marineris Hemisphere Enhanced,http://astropedia.astrogeology.usgs.gov/downlo...


<IPython.core.display.Javascript object>